# Clustering

In [ ]:
# Setup
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import os

PROJECT_ROOT = ".." 
DIRS = {
    'data':    os.path.join(PROJECT_ROOT, 'outputs', 'data'),
    'models':  os.path.join(PROJECT_ROOT, 'outputs', 'models'),
    'figures': os.path.join(PROJECT_ROOT, 'outputs', 'figures'),
    'results': os.path.join(PROJECT_ROOT, 'outputs', 'results')
}

In [ ]:
Z_pca = np.load(f"{DIRS['data']}/Z_pca_test.npy")
Z_ae = np.load(f"{DIRS['data']}/Z_ae_test.npy")
y_true = np.load(f"{DIRS['data']}/y_test.npy")

n_clusters = len(np.unique(y_true))
kmeans = KMeans(n_clusters, random_state=42)
labels_pca = kmeans.fit_predict(Z_pca)
labels_ae = kmeans.fit_predict(Z_ae)

results = []

algorithms = {
    'KMeans': KMeans(n_clusters=n_clusters, random_state=42, n_init=10),
    'Hierarchical': AgglomerativeClustering(n_clusters=n_clusters)
}

feature_sets = {
    'PCA': Z_pca,
    'Autoencoder': Z_ae
}

for feat_name, X_feat in feature_sets.items():
    for algo_name, model in algorithms.items():
        
        labels_pred = model.fit_predict(X_feat)
        
        ari_score = adjusted_rand_score(y_true, labels_pred)
        nmi_score = normalized_mutual_info_score(y_true, labels_pred)
        
        results.append({
            'Feature Set': feat_name,
            'Algorithm': algo_name,
            'ARI Score': ari_score,
            'NMI Score': nmi_score
        })
        print(f"Processed: {feat_name} + {algo_name}")


df = pd.DataFrame(results)
df = df.sort_values(by=['ARI Score', 'NMI Score'], ascending=False).reset_index(drop=True)

df.to_csv(f"{DIRS['results']}/clustering_metrics.csv", index=False)

print("\n--- Clustering Performance Summary ---")
print(df)

Processed: PCA + KMeans
Processed: PCA + Hierarchical
Processed: Autoencoder + KMeans
Processed: Autoencoder + Hierarchical

--- Clustering Performance Summary ---
   Feature Set     Algorithm  ARI Score  NMI Score
0          PCA        KMeans   0.381091   0.719343
1          PCA  Hierarchical   0.375957   0.724733
2  Autoencoder        KMeans   0.349860   0.704118
3  Autoencoder  Hierarchical   0.339173   0.714251
